In [2]:
import numpy as np
import scipy.io
import os
import pandas as pd
import math
from pathlib import Path
from mayavi import mlab
import copy

In [3]:
from IPython.core.debugger import set_trace

Set the root directory where you have your .mat files

## matlab_loader

This makes a data loader class for matlab files and also passes a filter to remove noise 

In [4]:
class matlab_loader:
    """ Loads .mat files from a direcotory 
            
            root_dir (str): path to the directory that containes files
            mat_file (str): name of file 
     
         returns:
             self.array (np.array)
             self.mat_file (Path object)
    """
    def __init__(self, root_dir, mat_file):  
        
        root_dir = Path(root_dir)
        mat_file = Path(mat_file)
        
        mat_obj = scipy.io.loadmat(root_dir/mat_file)
        obj= mat_obj.keys()
        obj = list(obj)
        array = mat_obj[obj[-1]]
        
        self.array = array
        self.mat_file = root_dir/mat_file
        
    def __repr__(self):
        return f'{self.mat_file}'

In [5]:
def xyz(arr, filter_level = 0.3):
    """Convert 3D voxel arry to xyz coordiates.
    
            arr (np.array): 3D voxel array    
            filter_level (int/float): sets the threshold level for what is considered a voxel 
            
            returns: 
                np.array (n x 3)
    """
    
    # Everything above filter level is converted to 1
    arr = np.where(arr < filter_level, 0, 1)
    
    x, y, z = np.where(arr == 1)
    
    # converts the xyz so z is is *up* 
    x -= arr.shape[1]
    y -= arr.shape[0]
    x *= -1
    y *= -1
    xyz = np.array([x, y, z]).T
    return xyz

## PCA

In [6]:
def pca(xyz):
    """PCA on a xyz points array
            xyz(np.array): n x 3 array of xyz coordinates
    """
    
    #covaraince of xyz points
    cov = np.cov(xyz.T)
    
    #eiganvalues and vectors
    (eig_val, eig_vec) = np.linalg.eig(cov)

    mean_x, mean_y, mean_z = [np.mean(xyz[:,0]),np.mean(xyz[:,1]),np.mean(xyz[:,2])]

    return mean_x, mean_y, mean_z, eig_val, eig_vec

## Classes:

### Bone

In [7]:
class bone:
        def __init__(self, array, filter_level=0.3):
          
            if array is None:
                pass
            
            else:
                self.array = array
                self.filter_level = filter_level
                self.xyz = xyz(array, filter_level)

                mean_x, mean_y, mean_z, eig_val, eig_vec = pca(self.xyz) 

                self.vec = eig_vec
                self.PC1 = eig_vec[:,0]
                self.PC2 = eig_vec[:,1]
                self.PC3 = eig_vec[:,2]   

                self.mean = (mean_x, mean_y, mean_z)

### Foot bone

### Indivual bone classes

In [9]:
class foot_bone(bone):
    def __init__(self, name = 'UN-NAMED', **kwargs):
        
        for key, value in kwargs.items():
            setattr(self, key, value)
            
        self.name = name
        
    def __repr__(self):
        return self.name

In [10]:
class n_ang(foot_bone):
    def __init__(self, n_ang_name, f1 = None ,f2 = None):
                  
        self.f1 = bone(f1)
        self.f2 = bone(f2)
        self.n_name = n_ang_name
        
    def __repr__(self):
        return f'{self.n_name} ' 

## Ploting:

In [12]:
def voxel_points(bone, color):
    mlab.points3d(bone[:,0],
                  bone[:,1],
                  bone[:,2], 
                  mode="cube", color=color)
    
def voxel_PCAs(bone,pc):
    pc_color = [(1,0,0),(0,1,0), (0,0,1)]
    
    for n in range(1,4):
        mlab.quiver3d(0,0,0, 
                      getattr(bone,f'{pc}{n}')[0],
                      getattr(bone,f'{pc}{n}')[1], 
                      getattr(bone,f'{pc}{n}')[2], 
                      line_width =6, scale_factor= 100, color= pc_color[n-1])
        

## bone_plot

In [13]:
# NEED TO REFACTOR
def bone_plot(*args, user_colours = None, plot_PCA = True, plot_inv = False):
    """plots voxel array; can take n bones and plot PCA vectors"""

    # Sorting out colours
    colour_dict = {'yellow':(0.9,0.9,0),
                   'pastel_blue':(0.7,1,1),
                   'purple':(0.6,0,0.5),
                   'orange':(0.8,0.3,0),
                   'dark_blue':(0,0.3,0.7),}
    
    if user_colours is None:
        user_colours = colour_dict
    
    plot_colours = []
    
    for col in user_colours:
        x = colour_dict.get(col)
        plot_colours.append(x)
        
    
    for n, bone in enumerate(args):

        mlab.points3d(bone.xyz[:,0], bone.xyz[:,1], bone.xyz[:,2],
                     mode="cube",
                     color= plot_colours[n],
                     scale_factor=1)
        
        x,y,z = bone.mean

        #plot princible vectors
        u0,v0,w0 = bone.PC1 * 100
        u0_inv,v0_inv,w0_inv = bone.PC1 * 100 * -1

        u1,v1,w1 = bone.PC2 * 100
        u1_inv,v1_inv,w1_inv = bone.PC2 * 100 * -1

        u2,v2,w2 = bone.PC3 * 100
        u2_inv,v2_inv,w2_inv = bone.PC3 * 100 * -1

        print(f"{n}th bone PCA vectors: \n {bone.vec} \n ")
        
        
        if plot_PCA is True:
            mlab.quiver3d(x,y,z,u0,v0,w0,
                                 line_width =6,
                                 scale_factor=0.7,
                                 color= (1,0,0))
            mlab.quiver3d(x,y,z,u1,v1,w1,
                                 line_width =6,
                                 scale_factor= 0.5,
                                 color= (0,1,0))
            mlab.quiver3d(x,y,z,u2,v2,w2, 
                                 line_width =6,
                                 scale_factor=0.3,
                                 color=(0,0,1))


        #ploting the inverse of eigen vectors
        if plot_inv is True:
            mlab.quiver3d(x,y,z,u0_inv,v0_inv,w0_inv,
                                 line_width =6,
                                 scale_factor=0.7,
                                 color= (1,0,0))
            mlab.quiver3d(x,y,z,u1_inv,v1_inv,w1_inv,
                                 line_width =6,
                                 scale_factor=0.5,
                                 color= (0,1,0))
            mlab.quiver3d(x,y,z,u2_inv,v2_inv,w2_inv,
                                 line_width =6,
                                 scale_factor=0.3,
                                 color=(0,0,1))

    return mlab.show()

In [14]:
# Option user colours
# user_colours=['yellow','purple']

In [15]:
# bone_plot(phantom_tibia_f1,phantom_tibia_f2)

## Angels

In [16]:
def mag(v):
    """Finds magnitude of vector
        v (np.array): vector
    
        returns: magnitude of vector (numeric)
    """
    return math.sqrt(np.dot(v,v))

#angel of vector
def angle(v1, v2):
    """Finds angel between vectors"""
    try:
        x = math.acos(np.dot(v1, v2) / (mag(v1) * mag(v2)))
    
    except:
        x = 0
        
    return x

In [17]:
root_dir = Path('C:\\Users\luke\OneDrive - University College London\Marta\data'); root_dir

WindowsPath('C:/Users/luke/OneDrive - University College London/Marta/data')

In [18]:
%%time
# load data
tibia_phant_f2 = matlab_loader(root_dir, mat_file = 'phantom/phantom_tibia_f2.mat' )
tibia_phant_f1 = matlab_loader(root_dir, mat_file = 'phantom/phantom_tibia_f1.mat')
tibia_3_f2 = matlab_loader(root_dir, mat_file = 'fista_recons/3 angles/tibia_f2.mat')
tibia_6_f2 = matlab_loader(root_dir, mat_file = 'fista_recons/6 angles/tibia_f2.mat')
tibia_10_f2 = matlab_loader(root_dir, mat_file = 'fista_recons/10 angles/tibia_f2.mat')
tibia_15_f2 = matlab_loader(root_dir, mat_file = 'fista_recons/15 angles/tibia_f2.mat')

Wall time: 1.88 s


In [19]:
#root_dir = Path('C:\\Users\luke\Downloads\low_res')

In [20]:
# tibia_phant_f2 = matlab_loader(root_dir, mat_file = 'talus_f2.mat')
# tibia_phant_f1 = matlab_loader(root_dir, mat_file = 'talus_f1.mat')
# tibia_3_f2 = matlab_loader(root_dir, mat_file = 'fista recons/3 angles/talus_f2.mat')
# tibia_6_f2 = matlab_loader(root_dir, mat_file = 'fista recons/6 angles/talus_f2.mat')

In [21]:
%%time
tibia = foot_bone(name = 'tibia')

tibia.phantom = n_ang(f1=tibia_phant_f1.array,
                      f2=tibia_phant_f2.array,
                      n_ang_name= 'phantom')

tibia.ang3 = n_ang(f2 = tibia_3_f2.array,
                   n_ang_name= 'angle 3')
    

# ang6(f2 = tibia_6_f2.array),
# ang10(f2 = tibia_10_f2.array),
# ang15(f2 = tibia_15_f2.array)

Wall time: 1.06 s


In [22]:
talus_phant_f2 = matlab_loader(root_dir, mat_file = 'phantom/phantom_talus_f2.mat' )
talus_phant_f1 = matlab_loader(root_dir, mat_file = 'phantom/phantom_talus_f1.mat')

talus = foot_bone(name = 'talus')

talus.phantom = n_ang(f1=talus_phant_f1.array,
                      f2=talus_phant_f2.array,
                      n_ang_name= 'phantom')

In [308]:
#bone_plot(talus.phantom.f1, talus.phantom.f2)

0th bone PCA vectors: 
 [[ 0.23452909 -0.94849613  0.2129582 ]
 [-0.95038858 -0.2697696  -0.15487383]
 [-0.20434688  0.16607062  0.96470871]] 
 
1th bone PCA vectors: 
 [[-0.60124372  0.66587279  0.44172324]
 [ 0.59393072  0.7422138  -0.31042708]
 [ 0.53455803 -0.07571066  0.84173369]] 
 


In [23]:
# have __iter__ so you can loop through PCAs
def df_angles(bone, ang3 = None, ang6 = None, ang10 = None, ang15 = None, ALL = None):
    
    df = pd.DataFrame()
    
    #Phant f1 vs Phant f2 
    df.loc[f'{bone} phantom f1: PC1' ,f'{bone} phantom f2: PC1'] = angle(bone.phantom.f1.PC1,bone.phantom.f2.PC1)
    df.loc[f'{bone} phantom f1: PC2' ,f'{bone} phantom f2: PC2'] = angle(bone.phantom.f1.PC2,bone.phantom.f2.PC2)
    df.loc[f'{bone} phantom f1: PC3' ,f'{bone} phantom f2: PC3'] = angle(bone.phantom.f1.PC3,bone.phantom.f2.PC3)
 
    if ang3 or ALL is True:
        #Phant f2 vs ang_3 f2
        df.loc[f'{bone} phantom f1: PC1' ,f'{bone} ang3 f2: PC1'] = angle(bone.phantom.f1.PC1,bone.ang3.f2.PC1)
        df.loc[f'{bone} phantom f1: PC2' ,f'{bone} ang3 f2: PC2'] = angle(bone.phantom.f1.PC2,bone.ang3.f2.PC2)
        df.loc[f'{bone} phantom f1: PC3' ,f'{bone} ang3 f2: PC3'] = angle(bone.phantom.f1.PC3,bone.ang3.f2.PC3)

    if ang6 or ALL is True:
        #Phant f2 vs ang_6 f2
        df.loc[f'{bone} phantom f1: PC1' ,f'{bone} ang6 f2: PC1'] = angle(bone.phantom.f1.PC1,bone.ang6.f2.PC1)
        df.loc[f'{bone} phantom f1: PC2' ,f'{bone} ang6 f2: PC2'] = angle(bone.phantom.f1.PC2,bone.ang6.f2.PC2)
        df.loc[f'{bone} phantom f1: PC3' ,f'{bone} ang6 f2: PC3'] = angle(bone.phantom.f1.PC3,bone.ang6.f2.PC3)
    
    if ang10 or ALL is True:
        #Phant f2 vs ang_10 f2
        df.loc[f'{bone} phantom f1: PC1' ,f'{bone} ang10 f2: PC1'] = angle(bone.phantom.f1.PC1,bone.ang10.f2.PC1)
        df.loc[f'{bone} phantom f1: PC2' ,f'{bone} ang10 f2: PC2'] = angle(bone.phantom.f1.PC2,bone.ang10.f2.PC2)
        df.loc[f'{bone} phantom f1: PC3' ,f'{bone} ang10 f2: PC3'] = angle(bone.phantom.f1.PC3,bone.ang10.f2.PC3)
    
    if ang10 or ALL is True:
        #Phant f2 vs ang_15 f2
        df.loc[f'{bone} phantom f1: PC1' ,f'{bone} ang10 f2: PC1'] = angle(bone.phantom.f1.PC1,bone.ang15.f2.PC1)
        df.loc[f'{bone} phantom f1: PC2' ,f'{bone} ang10 f2: PC2'] = angle(bone.phantom.f1.PC2,bone.ang15.f2.PC2)
        df.loc[f'{bone} phantom f1: PC3' ,f'{bone} ang10 f2: PC3'] = angle(bone.phantom.f1.PC3,bone.ang15.f2.PC3)
    
    return df

In [24]:
#tibia.ang10.f2.PC1 , tibia.phantom.f2.PC1
#point excption of same

In [25]:
#%%time
#df_angles(tibia, ALL = True)

In [26]:
# bone_plot(tibia.phantom.f1)

# Rotation

## Step 1: Center the 2 means

displacement of PC1 and PC1_pos2

then rotate around that vector the angels of PC2 and PC3

In [27]:
def voxel_center(bone):
    #moves f1 onto f2
    tfm =  np.asarray(bone.f1.mean) - np.asarray(bone.f2.mean)
    
    #changing bone matrix coords f1
    bone.f1.tfm_xyz = bone.f1.xyz + tfm
    
    #sets mean to origin
    for n, i in enumerate(bone.f1.mean):
         bone.f1.tfm_xyz[:,n] -= bone.f1.mean[n] 
    
    #changing bone matrix coords f2
    bone.f2.tfm_xyz = bone.f2.xyz.astype(np.float64)
    #sets mean to origin
    for n, i in enumerate(bone.f2.mean):
         bone.f2.tfm_xyz[:,n] -= bone.f1.mean[n] 
            
    return bone

In [28]:
voxel_center(tibia.phantom)

phantom 

## Quatertions

In [36]:
#import quaternion

In [29]:
from pyquaternion import Quaternion

In [219]:
# def rotation(bone_f1, bone_f2):
    
#     # PC1 
#     # angle
#     ang = angle(bone_f1, bone_f2)
    
#     # cross product
#     #x,y,z = np.cross(bone_f1, bone_f2)
#     # Quaternion around crossproduct
    
#     r = Quaternion(axis = np.cross(bone_f1, bone_f2), angle = ang)
    
#     for pc in bone_f1:
#         bone_f1.pc = r.rotate(bone_f1)

#         x = np.apply_along_axis(lambda x: r.rotate(x), 1, tfm_tib_f1)

In [220]:
# def rot3d(x,r):
#     np.apply_along_axis(r.rotate, 1, x)

## Rotaion around PCs

Have to invtert the vectors if they are going the wrong way or it won't work

In [30]:
def roation(bone):
    
    # init tfm_PCn
    for n in range(1,4):
        setattr(bone.f1,f'tfm_PC{n}',getattr(bone.f1,f'PC{n}'))
        
    # for each pc rotate    
    for n in range(1,4):
    
        f1_PCn = getattr(bone.f1,f'tfm_PC{n}')
        f2_PCn = getattr(bone.f2,f'PC{n}')
            
        # angle between PCs
        ang =  angle(f1_PCn, f2_PCn)
        
        #cross product between PCs
        x,y,z = np.cross(f1_PCn, f2_PCn)

        r = Quaternion(axis = [x,y,z], angle = ang)
        
        # rotate voxelx
        #set_trace()
        bone.f1.tfm_xyz = np.apply_along_axis(r.rotate, 1, bone.f1.tfm_xyz)
        print(f'{n} {r}')
        
        #rotate PCs
        for n in range(1,4):
            setattr(bone.f1,
                    f'tfm_PC{n}',
                    r.rotate(getattr(bone.f1,f'tfm_PC{n}')))
                                            
    return bone

In [31]:
def rotation_plot(bone):
    
   #plot rotated f1
    voxel_points(bone.f1.tfm_xyz,color=(0,0.7,0))
    
    # plot og f2 
    voxel_points(bone.f2.tfm_xyz,color=(0.7,0,0))

    #f2 PCA 
    voxel_PCAs(bone.f2,'PC')
    
#     mlab.quiver3d(0,0,0, bone.f2.PC1[0], bone.f2.PC1[1], bone.f2.PC1[2], line_width =6, scale_factor= 100, color= (1,0,0))
#     mlab.quiver3d(0,0,0,bone.f2.PC2[0], bone.f2.PC2[1], bone.f2.PC2[2], line_width =6, scale_factor= 100, color= (0,0,1))
#     mlab.quiver3d(0,0,0,bone.f2.PC3[0], bone.f2.PC3[1], bone.f2.PC3[2], line_width =6, scale_factor= 100, color= (0,1,0))
    

    #w/ rotion
    voxel_PCAs(bone.f1,'tfm_PC')
    
#     mlab.quiver3d(0,0,0, bone.f1.tfm_PC1[0], bone.f1.tfm_PC1[1], bone.f1.tfm_PC1[2], line_width =6, scale_factor= 100, color= (1,0,0))
#     mlab.quiver3d(0,0,0,bone.f1.tfm_PC2[0], bone.f1.tfm_PC2[1], bone.f1.tfm_PC2[2], line_width =6, scale_factor= 100, color= (0,0,1))
#     mlab.quiver3d(0,0,0,bone.f1.tfm_PC3[0], bone.f1.tfm_PC3[1], bone.f1.tfm_PC3[2], line_width =6, scale_factor= 100, color= (0,1,0))
    

    mlab.show()

In [32]:
roation(tibia.phantom)

1 +0.087 -0.854i -0.401j -0.320k
2 +0.651 -0.198i -0.150j +0.717k
3 +1.000 -0.000i +0.000j -0.000k


phantom 

In [33]:
rotation_plot(tibia.phantom)

# Testing

In [279]:
talus = foot_bone(name = 'talus')

talus.phantom = n_ang(f1=talus_phant_f1.array,
                      f2=talus_phant_f2.array,
                      n_ang_name= 'phantom')

In [280]:
talus.phantom =voxel_center(talus.phantom)

# ...pc1

In [281]:
# # inv module 
# pc1_inv = talus.phantom.f2.PC1*-1
# ang =  angle(talus.phantom.f1.PC1 ,pc1_inv)
# x,y,z = np.cross(talus.phantom.f1.PC1,pc1_inv)

In [282]:
ang =  angle(talus.phantom.f1.PC1 ,talus.phantom.f2.PC1)
x,y,z = np.cross(talus.phantom.f1.PC1,talus.phantom.f2.PC1)

In [284]:
r1 = Quaternion(axis = [x,y,z], angle = ang)

In [287]:
talus.phantom.f1.tfm_xyz = np.apply_along_axis(r1.rotate, 1, talus.phantom.f1.tfm_xyz)

In [288]:
pc1 = talus.phantom.f1.PC1
pc2 = talus.phantom.f1.PC2
pc3 = talus.phantom.f1.PC3

In [289]:
pc1= r1.rotate(pc1)
pc2= r1.rotate(pc2)
pc3= r1.rotate(pc3)

# ...pc2

In [290]:
#pc2_inv = tibia.phantom.f2.PC2*-1

In [291]:
#ang = (angle(pc2_inv ,tibia.phantom.f1.PC2))

In [292]:
ang = (angle(pc2 ,talus.phantom.f2.PC2))

In [293]:
ang

0.9365642423615254

In [294]:
x,y,z= np.cross(pc2,talus.phantom.f2.PC2)

In [295]:
r2 = Quaternion(axis = [x,y,z], angle = ang)

In [296]:
pc1 = r2.rotate(pc1)
pc2 = r2.rotate(pc2)
pc3 = r2.rotate(pc3)

In [297]:
talus.phantom.f1.tfm_xyz = np.apply_along_axis(r2.rotate, 1, talus.phantom.f1.tfm_xyz)

# ...pc3

In [298]:
#pc3_inv = tibia.phantom.f2.PC3*-1

In [299]:
#ang = (angle(pc3_inv ,tibia.phantom.f1.PC3))

In [300]:
ang =  angle(pc3,talus.phantom.f2.PC3)

In [302]:
x,y,z= np.cross(pc3, talus.phantom.f2.PC3)

In [303]:
r3 = Quaternion(axis = [x,y,z], angle = ang)

In [304]:
pc1 = r3.rotate(pc1)
pc2 = r3.rotate(pc2)
pc3 = r3.rotate(pc3)

In [305]:
talus.phantom.f1.tfm_xyz = np.apply_along_axis(r3.rotate, 1, talus.phantom.f1.tfm_xyz)

In [307]:
# roatated 
mlab.points3d(talus.phantom.f1.tfm_xyz[:,0],
              talus.phantom.f1.tfm_xyz[:,1],
              talus.phantom.f1.tfm_xyz[:,2], 
              mode="cube", color=(0,0.7,0))

#f2
mlab.points3d(talus.phantom.f2.tfm_xyz[:,0],
              talus.phantom.f2.tfm_xyz[:,1],
              talus.phantom.f2.tfm_xyz[:,2],
              mode="cube", color=(1,0,0))

#f2 PCA 
mlab.quiver3d(0,0,0,talus.phantom.f2.PC1[0],talus.phantom.f2.PC1[1],talus.phantom.f2.PC1[2], line_width =6, scale_factor= 100, color= (1,0,0))
mlab.quiver3d(0,0,0,talus.phantom.f2.PC2[0],talus.phantom.f2.PC2[1],talus.phantom.f2.PC2[2], line_width =6, scale_factor= 100, color= (0,0,1))
mlab.quiver3d(0,0,0,talus.phantom.f2.PC3[0],talus.phantom.f2.PC3[1],talus.phantom.f2.PC3[2], line_width =6, scale_factor= 100, color= (0,1,0))

#w/ rotion
mlab.quiver3d(0,0,0,pc1[0],pc1[1],pc1[2], line_width =6, scale_factor= 100, color= (0.6,0,0))
mlab.quiver3d(0,0,0,pc2[0],pc2[1],pc2[2], line_width =6, scale_factor= 100, color= (0,0,1))
mlab.quiver3d(0,0,0,pc3[0],pc3[1],pc3[2], line_width =6, scale_factor= 100, color= (0,1,0))

mlab.show()